In [ ]:
import boto3
from dotenv import load_dotenv
import os
import warnings
from io import StringIO
import pandas as pd
from tqdm import tqdm
import numpy as np
import io
import requests
from datetime import datetime


import sys
sys.path.append('..')
import utils
tqdm.pandas()

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
warnings.filterwarnings('ignore')

load_dotenv()
aws_access_key = os.getenv('AWS_ACCESS_KEY')
aws_secret_access = os.getenv('AWS_SECRET_ACCESS')
aws_region = os.getenv('AWS_REGION')

s3 = boto3.client('s3',
                aws_access_key_id=aws_access_key,
                aws_secret_access_key=aws_secret_access,
                region_name=aws_region)

bucket = 'footballbets'
league = "ENG-Premier League"
season = 2223

In [ ]:
scheduler = s3.get_object(Bucket=bucket, Key=f'ENG-Premier League/2223/schedule.csv')
schedule = pd.read_csv(StringIO(scheduler['Body'].read().decode('utf-8')))

In [ ]:
schedule.sort_values('start_time').head()

In [ ]:
schedule.start_time.dtype

In [ ]:
something = requests.get("https://fixturedownload.com/download/champions-league-2022-AtlanticStandardTime.csv")

In [ ]:
europe = pd.read_csv(io.StringIO(something.text))

europe['date'] = europe.Date.apply(lambda x: str(x).split(' ')[0])
europe['date'] = europe.date.apply(lambda x: datetime.strptime(x, "%d/%m/%Y").date())
europe['time'] = europe.Date.apply(lambda x: str(x).split(' ')[1])
europe['time'] = europe.time.apply(lambda x: datetime.strptime(x, "%H:%M").time())

In [ ]:
europe['league'] = 'Champions League'
europe['season'] = 2223

europe['game'] = europe.apply(lambda x: f"{x.date} {x['Home Team']}-{x['Away Team']}", axis=1)
europe['start_time'] = europe.apply(lambda x: f"{x.date}T{x.time}", axis=1)
europe = europe.rename(columns={'Home Team':'home_team', 'Away Team':'away_team'})

cols_to_keep = ['league', 'season', 'game', 'start_time', 'home_team', 'away_team']
nul_cols = schedule.columns.difference(cols_to_keep)

europe = europe.drop(europe.columns.difference(cols_to_keep), axis=1)
europe[nul_cols] = np.nan
europe.head()

In [ ]:
final_sched = pd.concat([schedule, europe], ignore_index=True).sort_values('start_time')

In [ ]:
final_sched.home_team.unique()